In [1]:
from neo4j import GraphDatabase
import os

In [2]:
AURA_CONNECTION_URI = os.getenv('NEO4J_URI')
AURA_USERNAME = os.getenv('NEO4J_USER')
AURA_PASSWORD = os.getenv('NEO4J_PASSWORD')

driver = GraphDatabase.driver(
    AURA_CONNECTION_URI,
    auth=(AURA_USERNAME, AURA_PASSWORD)
)

In [4]:
import json

with open('mock_data/food.json', 'r') as f:
    foods = json.load(f)

In [12]:
bag_of_words = {}
simple_ingredient = {}

for food, temp in foods.items():
    ingredients = temp['Ingredient']
    materials = [temp.split('(')[0].strip().lower() if '(' in temp else temp.strip().lower() for temp in ingredients]

    if food == "pad_thai": continue
    food = food.replace('_', ' ').lower()
    simple_ingredient[food] = materials
    
    for material in materials:
        if material in bag_of_words:
            bag_of_words[material] += 1
        else:
            bag_of_words[material] = 1

In [13]:
bag_of_words

{'mackerel': 1,
 'chili': 1,
 'garlic': 4,
 'shallots': 2,
 'shrimp paste': 1,
 'lime juice': 4,
 'vegetables': 1,
 'rice flour': 1,
 'coconut milk': 5,
 'sugar': 6,
 'salt': 2,
 'spring onions': 2,
 'chicken': 3,
 'turmeric powder': 1,
 'curry powder': 1,
 'peanut sauce': 1,
 'cucumber relish': 1,
 'fish paste': 1,
 'red curry paste': 1,
 'kaffir lime leaves': 3,
 'green beans': 1,
 'egg': 3,
 'fish sauce': 6,
 'grilled pork': 1,
 'chili powder': 1,
 'roasted rice powder': 1,
 'mint leaves': 1,
 'cilantro': 2,
 'wide rice noodles': 1,
 'soy sauce': 2,
 'chinese broccoli': 1,
 'pork or chicken': 1,
 'glutinous rice flour': 1,
 'palm sugar': 3,
 'grated coconut': 1,
 'water': 3,
 'soft tofu': 1,
 'ginger': 1,
 'massaman curry paste': 1,
 'beef': 1,
 'potatoes': 1,
 'onions': 1,
 'peanuts': 2,
 'cashew nuts': 1,
 'cinnamon': 1,
 'star anise': 1,
 'tamarind paste': 1,
 'shrimp': 1,
 'lemongrass': 1,
 'galangal': 1,
 "thai bird's eye chili": 2,
 'mushroom': 1,
 'green papaya': 1,
 'tamarin

In [14]:
similar_words = {}

for words in bag_of_words.keys():
    list_words = []
    for temp in bag_of_words.keys():
        if temp.lower() != words.lower() and words.lower() in temp.lower():
            list_words.append(temp.lower())
    
    if len(list_words) > 0:
        similar_words[words.lower()] = list_words

similar_words

{'chili': ['chili powder', "thai bird's eye chili"],
 'rice flour': ['glutinous rice flour'],
 'sugar': ['palm sugar'],
 'chicken': ['pork or chicken', 'chicken or shrimp'],
 'egg': ['thai eggplant'],
 'onions': ['spring onions'],
 'shrimp': ['shrimp paste', 'chicken or shrimp'],
 'tamarind': ['tamarind paste'],
 'onion': ['spring onions', 'onions'],
 'lime': ['lime juice', 'kaffir lime leaves'],
 'cucumber': ['cucumber relish']}

In [15]:
def add_food_with_ingredients(tx, food_name, ingredients):
    query = """
    CREATE (a:Food {name: $food_name})
    WITH a
    UNWIND $ingredients AS ingredient
    MERGE (b:Ingredient {name: ingredient}) 
    MERGE (b)-[:RECIPE]->(a)
    """
    tx.run(query, food_name=food_name, ingredients=ingredients)

# Iterating through food and ingredients
for food, ingredients in simple_ingredient.items():
    with driver.session() as session:
        session.write_transaction(add_food_with_ingredients, food, ingredients)

/tmp/ipykernel_15489/3250473487.py:14: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(add_food_with_ingredients, food, ingredients)


In [16]:
def add_ingredient_relations(tx, ingredient_term, ingredient_variants):
    query = """
    MATCH (a:Ingredient {name: $ingredient_term})
    UNWIND $ingredient_variants AS variant
    MERGE (b:Ingredient {name: variant})
    MERGE (a)-[:SYNONYM]->(b)  // Consider changing to a more appropriate relationship type if necessary
    """ 
    tx.run(query, ingredient_term=ingredient_term, ingredient_variants=ingredient_variants)

for ingredient_term, ingredient_variants in similar_words.items():
    with driver.session() as session:
        session.write_transaction(add_ingredient_relations, ingredient_term, ingredient_variants)

/tmp/ipykernel_15489/790334638.py:12: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(add_ingredient_relations, ingredient_term, ingredient_variants)


In [ ]:
driver.close()

Mock user data for recommendation system

In [151]:
import json

N = 500

users = [f"user_{i}" for i in range(N)]
with open('mock_data/food.json', 'r') as f: data = json.load(f)
foods = list(data.keys())

user_to_index = {user:i for i, user in enumerate(users)}
index_to_user = {i:user for i, user in enumerate(users)}

food_to_index = {user:i for i, user in enumerate(foods)}
index_to_food = {i:user for i, user in enumerate(foods)}

In [152]:
import numpy as np

Matrix = np.zeros((N, len(foods))).astype(int)
Matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(500, 15))

In [153]:
index_not_halal = []
for food in foods:
    ingredients = data[food]["Ingredient"]
    for ingredient in ingredients:
        if "pork" in ingredient.lower():
            print(food)
            index_not_halal.append(food_to_index[food])
            break

print(index_not_halal)

nam_tok_moo
pad_see_ew
[5, 6]


In [161]:
import random

number_islam = random.randint(1, 150)
print(f"Number of Islam: {number_islam}")

islam_users = []
for _ in range(number_islam):
    while True:
        index = random.randint(0, N - 1)
        user = f"user_{index}"
        if user not in islam_users:
            islam_users.append(user)
            break

print(islam_users)

Number of Islam: 97
['user_123', 'user_201', 'user_302', 'user_392', 'user_387', 'user_35', 'user_462', 'user_487', 'user_170', 'user_148', 'user_8', 'user_112', 'user_409', 'user_485', 'user_233', 'user_358', 'user_197', 'user_331', 'user_323', 'user_405', 'user_193', 'user_153', 'user_42', 'user_384', 'user_310', 'user_126', 'user_254', 'user_106', 'user_227', 'user_220', 'user_444', 'user_44', 'user_70', 'user_333', 'user_218', 'user_69', 'user_348', 'user_419', 'user_224', 'user_102', 'user_160', 'user_475', 'user_191', 'user_427', 'user_481', 'user_6', 'user_87', 'user_24', 'user_312', 'user_389', 'user_296', 'user_114', 'user_147', 'user_452', 'user_490', 'user_105', 'user_46', 'user_195', 'user_330', 'user_23', 'user_372', 'user_288', 'user_376', 'user_207', 'user_273', 'user_289', 'user_398', 'user_455', 'user_498', 'user_278', 'user_229', 'user_143', 'user_379', 'user_423', 'user_401', 'user_202', 'user_217', 'user_360', 'user_131', 'user_316', 'user_415', 'user_352', 'user_28

In [162]:
for user in islam_users:
    index = user_to_index[user]
    if user in islam_users:
        for food in foods:
            if food_to_index[food] in index_not_halal:
                Matrix[index][food_to_index[food]] -= 1

print(Matrix[123])

[ 0  0  0  0  0 -1 -1  0  0  0  0  0  0  0  0]


In [163]:
with open('mock_data/allergy.json') as f: data2 = json.load(f)
allergies = list(data2.keys())
allergies

['fish',
 'shellfish',
 'chicken',
 'soy',
 'egg',
 'peanut',
 'tree_nut',
 'coconut',
 'wheat',
 'dairy']

In [164]:
allergy_to_food = {allergy:[] for allergy in allergies}
for food in foods:
    ingredients = data[food]["Ingredient"]
    for ingredient in ingredients:
        for allergy in allergies:
            if ingredient in data2[allergy]["Ingredient"]:
                index = food_to_index[food]
                allergy_to_food[allergy].append(index)

print(allergy_to_food)

{'fish': [0, 4, 5, 10, 11, 12, 13], 'shellfish': [2, 10], 'chicken': [2, 3, 9, 13], 'soy': [2, 6, 12], 'egg': [2, 4, 6, 12], 'peanut': [2, 9, 11], 'tree_nut': [], 'coconut': [1, 3, 9, 13, 14], 'wheat': [], 'dairy': []}


In [165]:
number_per_allergy = 5
total_allergies = number_per_allergy * len(allergies)
print(f"Number of allergies: {total_allergies}")

allergy_users = []
for _ in range(total_allergies):
    while True:
        index = random.randint(0, N - 1)
        user = f"user_{index}"
        if user not in allergy_users:
            allergy_users.append(user)
            break

print(allergy_users)

Number of allergies: 50
['user_73', 'user_346', 'user_361', 'user_40', 'user_118', 'user_63', 'user_292', 'user_416', 'user_104', 'user_285', 'user_69', 'user_456', 'user_68', 'user_407', 'user_349', 'user_78', 'user_96', 'user_259', 'user_455', 'user_185', 'user_216', 'user_54', 'user_94', 'user_161', 'user_15', 'user_57', 'user_228', 'user_389', 'user_170', 'user_297', 'user_478', 'user_371', 'user_381', 'user_3', 'user_33', 'user_150', 'user_330', 'user_338', 'user_426', 'user_162', 'user_45', 'user_414', 'user_336', 'user_396', 'user_366', 'user_443', 'user_293', 'user_401', 'user_335', 'user_480']


In [166]:
group_people = [allergy_users[i:i+ number_per_allergy] for i in range(0, total_allergies, number_per_allergy)]
for i, group in enumerate(group_people):
    for user in group:
        user_index = user_to_index[user]
        for index in list(allergy_to_food.items())[i][1]:
            Matrix[user_index][index] -= 1

print(Matrix[346])

[-1  0  0  0 -1 -1  0  0  0  0 -1 -1 -1 -1  0]


In [167]:
for user in users:
    i = user_to_index[user]
    food_list = Matrix[i]
    for index in range(len(food_list)):
        found = random.choice([True, False])
        if found:
            frequency = food_list[index]
            random_freq = random.randint(1, 10)
            if frequency == 0: food_list[index] = random_freq

print(Matrix[123])
print(Matrix[346])

[ 0  8  0  9  0 -1 -1  0  0  6  5 10  3  0  0]
[-1  6  0  0 -1 -1  2  8  4  6 -1 -1 -1 -1  0]


In [168]:
Matrix

array([[ 9,  4,  8, ...,  0,  2,  0],
       [ 0,  9,  2, ...,  0,  0,  0],
       [ 3,  2,  0, ...,  0,  1,  8],
       ...,
       [ 0, 10,  0, ..., 10,  0,  9],
       [ 3,  5,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  3, 10,  0]], shape=(500, 15))

In [169]:
np.save('mock_data/Matrix.npy', Matrix)

Recommend system

In [221]:
from sklearn.decomposition import TruncatedSVD
import pandas as pd
from transector import Transector
import random

Matrix = np.load('mock_data/Matrix.npy')
with open('mock_data/food.json', 'r') as f: data = json.load(f)
foods = list(data.keys())

def best_restaurant(top_foods):
    top_food_names = []
    with open('mock_data/food.json', 'r') as f:
        data = json.load(f)
        for food in top_foods:
            if food in data and "English" in data[food] and data[food]["English"]:
                top_food_names.append(data[food]["English"])

    with open('mock_data/resturant.json', 'r') as f:
        data2 = json.load(f)

    # Calculate restaurant scores
    scores = {}
    for k, v in data2.items():
        scores[v["name"]] = 0
        for m in v["menu"]:
            dish = m["dish"]
            if dish in top_food_names:
                scores[v["name"]] += 1

    # Handle cases where no matching restaurants are found
    if not scores:
        return {"error": "No matching restaurants found"}

    best_restaurant_name = max(scores, key=scores.get)
    best_restaurant = None
    for k, v in data2.items():
        if v["name"] == best_restaurant_name:
            best_restaurant = {"name": v["name"], "menu": v["menu"]}
            break

    return {"best_restaurant": best_restaurant}


def popularity():
    temp = [0 for _ in range(len(foods))]
    for j in range(len(foods)):
        for i in range(Matrix.shape[0]):
            frequency = Matrix[i][j]
            temp[j] += frequency

    # Get the indices of the top 5 most popular foods
    top_indices = np.argsort(temp)[-5:][::-1]
    top_foods = [foods[i] for i in top_indices]
    return best_restaurant(top_foods)

def CF(food_name):
    index = foods.index(food_name)

    # SVD to make correlation matrix
    SVD = TruncatedSVD(n_components= 10)
    decomposed_matrix = SVD.fit_transform(Matrix)
    correlation_matrix = pd.DataFrame(data= decomposed_matrix).corr()
    
    recommend_list = correlation_matrix[index].to_list()
    score_dict = {foods[i]: score for i, score in enumerate(recommend_list)} 

    filtered_scores = {k: v for k, v in score_dict.items() if k != food_name}
    top_foods = [k for k, v in sorted(filtered_scores.items(), key=lambda item: item[1], reverse=True)[:5]]
    return best_restaurant(top_foods)

def CB():
    transector = Transector()
    number = random.randint(0, 499)
    preference = Matrix[number]
    food_for_person = [foods[i] for i, freq in enumerate(preference) if freq > 0]

    set_ingredient = set()
    for food in food_for_person:
        ingredients = transector.get_ingredients_for_food(food)
        set_ingredient.update(ingredients)


    all_food_scores = {}
    for ingredient in set_ingredient:
        query_foods = transector.get_foods_for_ingredient(ingredient)

        for food in query_foods:
            if food not in food_for_person:
                food_ingredients = transector.get_ingredients_for_food(food)

                # Calculate similarity score based on common ingredients
                common_ingredients = set_ingredient.intersection(food_ingredients)
                similarity_score = len(common_ingredients) / len(set_ingredient.union(food_ingredients))

                if food in all_food_scores:
                    all_food_scores[food] += similarity_score
                else:
                    all_food_scores[food] = similarity_score


    recommended_foods = sorted(all_food_scores.items(), key=lambda x: x[1], reverse=True)
    top_food = [food for food, score in recommended_foods[:5]]
    transector.close()

    return best_restaurant(top_food)

In [222]:
print(popularity())

{'best_restaurant': {'name': 'Thai Royal Kitchen', 'menu': [{'dish': 'Chili Dip with Fried Mackerel', 'price': '227.50 THB'}, {'dish': 'Massaman Curry', 'price': '332.50 THB'}, {'dish': 'Stir-Fried Rice Noodles', 'price': '245.00 THB'}, {'dish': 'Grilled Chicken Skewers', 'price': '262.50 THB'}, {'dish': 'Tofu Pudding in Ginger Syrup', 'price': '192.50 THB'}]}}


In [223]:
print(CF(food_name="massaman"))

{'best_restaurant': {'name': 'Bangkok Flavors', 'menu': [{'dish': 'Spicy Grilled Pork Salad', 'price': '297.50 THB'}, {'dish': 'Stir-Fried Wide Rice Noodles', 'price': '245.00 THB'}, {'dish': 'Coconut Dumplings', 'price': '140.00 THB'}, {'dish': 'Chili Dip with Fried Mackerel', 'price': '210.00 THB'}, {'dish': 'Massaman Curry', 'price': '350.00 THB'}]}}


In [224]:
print(CB())

{'best_restaurant': {'name': 'Thai Royal Kitchen', 'menu': [{'dish': 'Chili Dip with Fried Mackerel', 'price': '227.50 THB'}, {'dish': 'Massaman Curry', 'price': '332.50 THB'}, {'dish': 'Stir-Fried Rice Noodles', 'price': '245.00 THB'}, {'dish': 'Grilled Chicken Skewers', 'price': '262.50 THB'}, {'dish': 'Tofu Pudding in Ginger Syrup', 'price': '192.50 THB'}]}}
